In [1]:
# This is a simple python script to print "Hello World" to the console
print("hello World")

Hola


In [5]:
!pip install scipy

     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ------------------------- ------------ 41.0/60.4 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 60.4/60.4 kB 796.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
   ---------------------------------------- 0.5/46.2 MB 15.5 MB/s eta 0:00:03
    --------------------------------------- 1.0/46.2 MB 13.1 MB/s eta 0:00:04
   - -------------------------------------- 1.6/46.2 MB 12.6 MB/s eta 0:00:04
   - -------------------------------------- 2.1/46.2 MB 12.3 MB/s eta 0:00:04
   -- ------------------------------------- 2.7/46.2 MB 12.3 MB/s eta 0:00:04
   -- ------------------------------------- 3.2/46.2 MB 12.1 MB/s eta 0:00:04
   --- ------------------------------------ 3.8/46.2 MB 12.1 MB/s eta 0:00:04
   --- ------------------------------------ 4.3/46.2 MB 12.0 MB/s eta 0:00:04
   ---- ----------------------------------- 4.9/46.2 MB 12.1 MB/s eta 0:00

In [16]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

# Step 1: Create a simulated dataset
users = ['User1', 'User2', 'User3', 'User4', 'User5']
movies = ['Movie1', 'Movie2', 'Movie3', 'Movie4', 'Movie5']

data = {
    'User1': [5, 4, 1, 2, 3],
    'User2': [4, 5, 1, 2, 3],
    'User3': [3, 4, 5, 1, 2],
    'User4': [2, 3, 4, 5, 1],
    'User5': [1, 2, 3, 4, 5],
}

df = pd.DataFrame(data, index=movies)

# Step 2: Define a function to calculate similarity between users
def calculate_similarity(dataframe):
    """Calculate the distance between each sample"""
    return pd.DataFrame(1/(1 + squareform(pdist(dataframe.T, 'euclidean'))), 
                        index=dataframe.columns, columns=dataframe.columns)

# Step 3: Predict a user's rating for a movie they haven't seen
def predict_ratings(similarity, dataframe):
    return similarity.dot(dataframe.T) / similarity.sum()

# Step 4: Recommend movies
def recommend_movies(predictions, user, movies, original_ratings, num_recommendations):
    # Get and sort the user's predictions
    sorted_user_predictions = predictions[user].sort_values(ascending=False)
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.user == user]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )

    print('User {0} has already rated {1} movies.'.format(user, user_full.shape[0]))
    print('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

similarity = calculate_similarity(df)
predictions = predict_ratings(similarity, df.T)

user = 'User1'
movies = pd.DataFrame(index=df.index, columns=['movieId'])
movies['movieId'] = range(1, len(df.index) + 1)
original_ratings = df.stack().reset_index()
original_ratings.columns = ['movieId', 'user', 'rating']

# Create a dictionary to map movie names to IDs
movie_to_id = movies.reset_index().set_index('index')['movieId'].to_dict()

# Map the movie names in original_ratings to their IDs
original_ratings['movieId'] = original_ratings['movieId'].map(movie_to_id)

already_rated, predictions = recommend_movies(predictions, user, movies, original_ratings, 5)
print("Already rated:")
print(already_rated)
print("\nRecommendations:")
print(predictions)

ValueError: matrices are not aligned

In [17]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

# Step 1: Create a simulated dataset
users = ['User1', 'User2', 'User3', 'User4', 'User5']
movies = ['Movie1', 'Movie2', 'Movie3', 'Movie4', 'Movie5']

data = {
    'User1': [5, 4, 1, 2, 3],
    'User2': [4, 5, 1, 2, 3],
    'User3': [3, 4, 5, 1, 2],
    'User4': [2, 3, 4, 5, 1],
    'User5': [1, 2, 3, 4, 5],
}

df = pd.DataFrame(data, index=movies)

# Step 2: Define a function to calculate similarity between users
def calculate_similarity(dataframe):
    """Calculate the distance between each sample"""
    return pd.DataFrame(1/(1 + squareform(pdist(dataframe.T, 'euclidean'))), 
                        index=dataframe.columns, columns=dataframe.columns)

# Step 3: Predict a user's rating for a movie they haven't seen
def predict_ratings(similarity, dataframe):
    return similarity.dot(dataframe.T) / similarity.sum()

# Step 4: Recommend movies
def recommend_movies(predictions, user, movies, original_ratings, num_recommendations):
    # Get and sort the user's predictions
    sorted_user_predictions = predictions[user].sort_values(ascending=False)
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings['user'] == user]  # Adjusted to match column name
    user_full = (user_data.merge(movies, how='left', left_on='movieId', right_index=True).  # Adjusted merge condition
                 sort_values(['rating'], ascending=False)
                 )

    print('User {0} has already rated {1} movies.'.format(user, user_full.shape[0]))
    print('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies.index.isin(user_full['movieId'])].  # Adjusted to match index
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how='left',
               left_on='movieId',
               right_index=True).  # Adjusted to match index
         rename(columns={user: 'Predictions'}).
         sort_values('Predictions', ascending=False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

similarity = calculate_similarity(df)
predictions = predict_ratings(similarity, df.T)

user = 'User1'
movies = pd.DataFrame(index=df.index, columns=['movieId'])
movies['movieId'] = range(1, len(df.index) + 1)
original_ratings = df.stack().reset_index()
original_ratings.columns = ['movieId', 'user', 'rating']

# Create a dictionary to map movie names to IDs
movie_to_id = movies.reset_index().set_index('index')['movieId'].to_dict()

# Map the movie names in original_ratings to their IDs
original_ratings['movieId'] = original_ratings['movieId'].map(movie_to_id)

already_rated, recommendations = recommend_movies(predictions, user, movies, original_ratings, 5)
print("Already rated:")
print(already_rated)
print("\nRecommendations:")
print(recommendations)


ValueError: matrices are not aligned

In [24]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

# Step 1: Create a simulated dataset
users = ['User1', 'User2', 'User3', 'User4', 'User5']
movies = ['Movie1', 'Movie2', 'Movie3', 'Movie4', 'Movie5']

data = {
    'User1': [5, 4, 1, 2, 3],
    'User2': [4, 5, 1, 2, 3],
    'User3': [3, 4, 5, 1, 2],
    'User4': [2, 3, 4, 5, 1],
    'User5': [1, 2, 3, 4, 5],
}

df = pd.DataFrame(data, index=movies)

# Step 2: Define a function to calculate similarity between users


def calculate_similarity(dataframe):
    """Calculate the distance between each sample"""
    return pd.DataFrame(1/(1 + squareform(pdist(dataframe.T, 'euclidean'))),
                        index=dataframe.columns, columns=dataframe.columns)

# Step 3: Predict a user's rating for a movie they haven't seen


def predict_ratings(similarity, dataframe):
    return similarity.dot(dataframe) / similarity.sum()

# Step 4: Recommend movies


def recommend_movies(predictions, user, movies, original_ratings, num_recommendations):
    # Get and sort the user's predictions
    sorted_user_predictions = predictions[user].sort_values(ascending=False)

    # Get the user's data and merge in the movie information.
    # Adjusted to match column name
    user_data = original_ratings[original_ratings['user'] == user]
    user_full = (user_data.merge(movies, how='left', left_on='movieId', right_index=True).  # Adjusted merge condition
                 sort_values(['rating'], ascending=False)
                 )

    print('User {0} has already rated {1} movies.'.format(
        user, user_full.shape[0]))
    print('Recommending highest {0} predicted ratings movies not already rated.'.format(
        num_recommendations))

    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies.index.isin(user_full['movieId'])].  # Adjusted to match index
                       merge(pd.DataFrame(sorted_user_predictions).reset_index(), how='left',
                             left_on='movieId',
                             right_index=True).  # Adjusted to match index
                       rename(columns={user: 'Predictions'}).
                       sort_values('Predictions', ascending=False).
                       iloc[:num_recommendations, :-1]
                       )

    return user_full, recommendations


similarity = calculate_similarity(df)
predictions = predict_ratings(similarity, df)

user = 'User1'
movies = pd.DataFrame(index=df.index, columns=['movieId'])
movies['movieId'] = range(1, len(df.index) + 1)
original_ratings = df.stack().reset_index()
original_ratings.columns = ['movieId', 'user', 'rating']

# Create a dictionary to map movie names to IDs
movie_to_id = movies.reset_index().set_index('index')['movieId'].to_dict()

# Map the movie names in original_ratings to their IDs
original_ratings['movieId'] = original_ratings['movieId'].map(movie_to_id)

already_rated, recommendations = recommend_movies(
    predictions, user, movies, original_ratings, 5)
print("Already rated:")
print(already_rated)
print("\nRecommendations:")
print(recommendations)

ValueError: matrices are not aligned